# 7.1 Group Invariance (Quantitative)

## Measurement invariance across groups for quantitative item responses

In [ ]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Rpy2 imports
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri, numpy2ri
from rpy2.robjects.packages import importr

# Automatic conversion of arrays and dataframes
pandas2ri.activate()
numpy2ri.activate()

# Set random seed for reproducibility
ro.r('set.seed(123)')

# Ipython extenrsion for magix plotting
%load_ext rpy2.ipython

# R imports
importr('base')
importr('lavaan')
importr('semTools')
importr('MPsychoR')
importr('lordif')
importr('Hmisc')

rpy2.robjects.packages.Package as a <module 'Hmisc'>

Measurement invariance means that we measure the same psychological construct across groups and time (see lectures 8 & 9 for examples). In the following RMD we will look at how we can assess measurement invariance across...

* **groups** for **quantitative** item responses
* **groups** for **qualitative** item responses
* **time** for **quantitative** responses  

The four most important measurement invariance structures are the following:

* **Configural invariance:** same factor structure but unrestricted loadings and intercepts across groups.
* **Metric invariance:** the factor structure and the loadings are constrained to be equal across groups; the intercepts are free. (In the book this is referred to as *Weak invariance*)
* **Scale invariance:** the factor structure, the loadings and the indicator intercepts are restricted to be equal across groups. (In the book this is referred to as *Strong invariance*)
* **Strict invariance:** the factor structure, the loadings, the indicator intercepts and the reliabilities are restricted to be equal across groups.

Note. In the book there's is also a model in which, in addition to the indicator intercepts, the means of the latent
variables are restricted to be equal as well, however we won't cover this here.  

Try to load and inspect the dataset. It's called `Bergh`.

### Load, prepare and inspect the dataset

In [3]:
ro.r('data("Bergh")')
# Convert to Python
Bergh = pandas2ri.rpy2py(ro.globalenv['Bergh'])
Bergh.head()

,EP,SP,HP,DP,A1,A2,A3,O1,O2,O3,gender
1,2.666667,3.125,1.4,2.818182,3.4375,3.600000,3.352941,2.8750,3.400000,3.176471,1
2,2.666667,3.250,1.4,2.545455,2.3125,2.666667,3.117647,4.4375,3.866667,4.470588,1
3,1.000000,1.625,2.7,2.000000,3.5625,4.600000,3.941176,4.2500,3.666667,3.705882,1
4,2.666667,2.750,1.8,2.818182,2.7500,3.200000,3.352941,2.8750,3.400000,3.117647,1
5,2.888889,3.250,2.7,3.000000,3.2500,4.200000,3.764706,3.9375,4.400000,4.294118,1


#### The dataset

We illustrate todays topic using a dataset from Bergh et al. (2016). Part of their
analysis consisted of a multigroup CFA, where ethnic prejudice *(EP)*, sexism *(SP)*,
sexual prejudice against gays and lesbians *(HP)*, and prejudice toward mentally
people with disabilities (DP) were modeled *as indicators of* a generalized prejudice
factor *(GP)*. The multigroup aspect comes in by exploring differences in CFA parameters across *gender*.

### Fit the models

Luckily, we don't have to specify all the models mentioned above by ourselves. Instead, we can use the `measurementInvariance()` function from the `semTools` package. We only need to specify few elements:

1. )  The model equation (i.e. which items we want to include),
2. )  The dataset,
3. )  The grouping variable and
4. )  The estimator (We use a robust ML estimator since some of the prejudice measures deviate from normality).  

Note. `strict = T` gives us a configural, metric (weak), scale (strong), strict and a fifth model. With `strict = F` we won't get the strict model.

In [4]:
ro.r("GP_model <- 'GP =~ EP + HP + DP + SP'")
ro.r('minvfit <- measurementInvariance(model = GP_model, data = Bergh, group = "gender", estimator = "MLR", strict = T)')


Measurement invariance models:

Model 1 : fit.configural
Model 2 : fit.loadings
Model 3 : fit.intercepts
Model 4 : fit.residuals
Model 5 : fit.means


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
               Df    AIC    BIC    Chisq Chisq diff Df diff Pr(>Chisq)    
fit.configural  4 7376.1 7490.2   1.6552                                  
fit.loadings    7 7382.9 7482.8  14.4960     11.834       3   0.007972 ** 
fit.intercepts 10 7417.7 7503.3  55.2875     47.833       3  2.311e-10 ***
fit.residuals  14 7429.8 7496.4  75.3689     12.468       4   0.014189 *  
fit.means      15 7482.4 7544.3 130.0375     49.267       1  2.234e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Fit measures:

       

This code chunk fits four invariance models: configural, metric (weak), scale (strong), strict and a fifth model where, in addition to the indicator intercepts, the means of the latent variables are restricted to be equal as well. As mentioned above, we won't deal with the last model.  

Each model is tested against the previous one, and a **significant result** indicates that the higher restricted model is significantly **worse** than the previous model. In our example, only configural variance holds as restricting the model parameters to match the assumptions of metric invariance results in a significantly worse fit.

#### Select the best fitting model

In our example we would go with the configural model, for which the output can be requested as follows:

In [5]:
# Extract only the configural model
ro.r("Fit_conf <- minvfit$fit.configural")

Try to investigate the `Fit_conf` using `summary()`. Remember to set `standardized = TRUE` & `fit.measures = TRUE`.

In [6]:
summary_conf = ro.r("summary(Fit_conf, standardized = TRUE, fit.measures = TRUE)")
print(summary_conf)

lavaan 0.6-19 ended normally after 45 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        24

  Number of observations per group:                   
    male                                           249
    female                                         612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                                 1.655       1.549
  Degrees of freedom                                 4           4
  P-value (Chi-square)                           0.799       0.818
  Scaling correction factor                                  1.068
    Yuan-Bentler correction (Mplus variant)                       
  Test statistic for each group:
    male                                         0.348       0.348
    female                                       1.201       1.201

Model Test Baseline Model

For the interpretation of this output, please refer to the previous sessions.

### Another Option

Instead of using these generic invariance sequences of models, we can approach
multigroup CFA from a more hypothesis-driven direction. In the first model we are
going to fit, the loadings for ethnic minorities (EP), gays/lesbians (HP), and people
with disabilities (DP) are held equal across men and women. We keep the sexism
**(SP) loadings free**. Note that all intercepts are free to vary.  

In `lavaan`, this can be achieved by using one of the following **two equivalent model formulations**.
In the first variant, we explicitly incorporate the loadings
restrictions in the model formulation. We specify a vector of length 2, the first
element denoting the loading for the first gender category and the second element
denoting the loading for the second category. Since we use the same loadings
symbols for both categories, they are restricted to be equal.

#### Option 1 to specify the model

In [7]:
# First option to specify the model # CHECK: The values align with R but the readability not so much...
ro.r ("GP_model <-'GP =~ c(v1,v1)*EP + c(v2,v2)*HP + c(v3,v3)*DP + SP'") # The loading of SP is not fixed.
ro.r('fitBase <-lavaan::cfa(GP_model, data = Bergh, group = "gender", estimator = "MLR")')
ro.r("fitMeasures(fitBase)")

array([ 2.20000000e+01,  4.25165558e-03,  7.32135091e+00,  6.00000000e+00,
        2.92148336e-01,  6.85411314e+00,  6.00000000e+00,  3.34549818e-01,
        1.06816896e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.98124627e-01,  9.96249254e-01,  9.98352222e-01,  9.96704444e-01,
        9.98697343e-01,  9.97394686e-01,  9.96249254e-01,  9.79565860e-01,
        9.89782930e-01,  4.94891465e-01,  9.98140462e-01,  9.98124627e-01,
        9.96704444e-01,  9.74152122e-01,  9.87076061e-01,  4.93538031e-01,
        9.98371078e-01,  9.98352222e-01,  9.97394686e-01,  9.98697343e-01,
       -3.66686045e+03, -3.66319978e+03,  7.37772090e+03,  7.48239898e+03,
        8.61000000e+02,  7.41253284e+03,  1.25325348e+00,  1.19508682e+00,
        2.26176188e-02,  0.00000000e+00,  6.94429738e-02,  9.00000000e-01,
        7.89925895e-01,  5.00000000e-02,  1.69311900e-02,  8.00000000e-02,
        1.81842544e-02,  

#### Option 2 to specify the model

In the second variant, we restrict all the loadings to be equal across groups
using the `group.equal` argument. We then free up the SP loading through the
`group.partial` argument.


In [8]:
# Second option to specify the model
ro.r("GP_model <- 'GP =~ EP + HP + DP + SP'")
ro.r('fitBase <- lavaan::cfa(GP_model,data = Bergh, group = "gender", group.equal = c("loadings"), group.partial = c("GP=~ SP"), estimator = "MLR")')
ro.r("fitMeasures(fitBase)")

array([ 2.20000000e+01,  4.25165558e-03,  7.32135091e+00,  6.00000000e+00,
        2.92148336e-01,  6.85411314e+00,  6.00000000e+00,  3.34549818e-01,
        1.06816896e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.98124627e-01,  9.96249254e-01,  9.98352222e-01,  9.96704444e-01,
        9.98697343e-01,  9.97394686e-01,  9.96249254e-01,  9.79565860e-01,
        9.89782930e-01,  4.94891465e-01,  9.98140462e-01,  9.98124627e-01,
        9.96704444e-01,  9.74152122e-01,  9.87076061e-01,  4.93538031e-01,
        9.98371078e-01,  9.98352222e-01,  9.97394686e-01,  9.98697343e-01,
       -3.66686045e+03, -3.66319978e+03,  7.37772090e+03,  7.48239898e+03,
        8.61000000e+02,  7.41253284e+03,  1.25325348e+00,  1.19508682e+00,
        2.26176188e-02,  0.00000000e+00,  6.94429738e-02,  9.00000000e-01,
        7.89925895e-01,  5.00000000e-02,  1.69311900e-02,  8.00000000e-02,
        1.81842544e-02,  

Both variants lead to the same results. Using the `fitMeasures(fitBase)`
call, we get a $\chi^2$-value of 7.321 (df = 6, p = 0.292), a RMSEA of 0.023 with a 90%
CI of [0, 0.069], a CFI of 0.998, and an SRMR of 0.028. The model fits well.  

Since the difference in intercepts between men and women is negligible for the
EP indicator, we can force the intercepts to be equal for this indicator. Let us build on
the second specification variant from above, in order to set up this model. Through
group.equal we **constrain all loadings and intercepts to be equal across groups**
and, subsequently, **free up the SP loading as well as the intercepts for DP, HP, and**
**SP** using `group.partial`:

In [9]:
ro.r("GP_model <- 'GP =~ EP + HP + DP + SP'")

ro.r('fitBase1 <- lavaan::cfa(GP_model, data = Bergh,\n'
      'group = "gender", group.equal = c("loadings", "intercepts"),\n'
      'group.partial = c("GP=~SP", "DP~1", "HP~1", "SP~1"),\n'
      'estimator = "MLR")')

ro.r("fitMeasures(fitBase1)")


array([ 2.20000000e+01,  4.25165558e-03,  7.32135091e+00,  6.00000000e+00,
        2.92148336e-01,  6.85411326e+00,  6.00000000e+00,  3.34549806e-01,
        1.06816894e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.98124627e-01,  9.96249254e-01,  9.98352222e-01,  9.96704444e-01,
        9.98697343e-01,  9.97394685e-01,  9.96249254e-01,  9.79565860e-01,
        9.89782930e-01,  4.94891465e-01,  9.98140462e-01,  9.98124627e-01,
        9.96704444e-01,  9.74152122e-01,  9.87076061e-01,  4.93538030e-01,
        9.98371077e-01,  9.98352222e-01,  9.97394685e-01,  9.98697343e-01,
       -3.66686045e+03, -3.66319978e+03,  7.37772090e+03,  7.48239898e+03,
        8.61000000e+02,  7.41253284e+03,  1.25325348e+00,  1.14728335e+00,
        2.26176188e-02,  0.00000000e+00,  6.94429738e-02,  9.00000000e-01,
        7.89925895e-01,  5.00000000e-02,  1.69311900e-02,  8.00000000e-02,
        1.81842557e-02,  

Again, this model fits well. We now use this model as **baseline model** and
**compare it to two additional models**. First **(1)**, we **constrain the SP loading to be 0**
for the **women** (ingroup-outgroup model).We can specify this restriction directly in
the model specification through `c(NA,0)`. *NA means free to vary* across men (first
group), whereas the second element fixes the parameter to 0 for the women. Note
that this is quite a restrictive assumption.

In [10]:
ro.r("GP_model2 <- 'GP =~ c(v1,v1)*EP + c(v2,v2)*HP + c(v3,v3)*DP + c(NA, 0)*SP'")

ro.r('fitIO <- lavaan::cfa(GP_model2, data = Bergh,\n'
      'group = "gender", group.equal = c("intercepts"),\n'
      'group.partial = c("DP~1", "HP~1", "SP~1"),\n'
      'estimator = "MLR")')

ro.r("fitMeasures(fitIO)")


array([ 2.10000000e+01,  1.35746530e-01,  2.33755525e+02,  7.00000000e+00,
        0.00000000e+00,  1.98735302e+02,  7.00000000e+00,  0.00000000e+00,
        1.17621541e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        6.78169360e-01,  4.48290332e-01,  6.30099155e-01,  3.65884267e-01,
        6.77994193e-01,  4.47990045e-01,  4.48290332e-01,  4.40783170e-01,
        6.73790183e-01,  3.93044273e-01,  6.80437114e-01,  6.78169360e-01,
        3.65884267e-01,  3.57605443e-01,  6.25269842e-01,  3.64740741e-01,
        6.33633179e-01,  6.30099155e-01,  4.47990045e-01,  6.77994193e-01,
       -3.78007754e+03, -3.66319978e+03,  7.60215508e+03,  7.70207506e+03,
        8.61000000e+02,  7.63538466e+03,  1.25325348e+00,  1.11906623e+00,
        2.74311074e-01,  2.44736269e-01,  3.05059830e-01,  9.00000000e-01,
        0.00000000e+00,  5.00000000e-02,  1.00000000e+00,  8.00000000e-02,
        2.52240862e-01,  

It gives a $\chi^2$-value of 233.756 (df = 7, p = 0), a RMSEA of 0.274 with a 90%
CI of [0.245, 0.305], a CFI of 0.678, and an SRMR of 0.148. Bad fit.  

In the next model **(2)**, we **restrict all the loadings to be equal** (**marginalization**
**model**). The intercepts have the same constraints as above.

In [11]:
ro.r('fitMarg <- lavaan::cfa(GP_model, data = Bergh,\n'
      'group = "gender", group.equal = c("loadings", "intercepts"),\n'
      'group.partial = c("DP~1", "HP~1", "SP~1"),\n'
      'estimator = "MLR")')

ro.r("fitMeasures(fitMarg)")


array([ 2.10000000e+01,  8.41811186e-03,  1.44959886e+01,  7.00000000e+00,
        4.30310073e-02,  1.34775671e+01,  7.00000000e+00,  6.12928392e-02,
        1.07556419e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.89361058e-01,  9.81761814e-01,  9.87503305e-01,  9.78577095e-01,
        9.90052292e-01,  9.82946787e-01,  9.81761814e-01,  9.65321030e-01,
        9.79770601e-01,  5.71532851e-01,  9.89436025e-01,  9.89361058e-01,
        9.78577095e-01,  9.56434938e-01,  9.74587047e-01,  5.68509111e-01,
        9.87622698e-01,  9.87503305e-01,  9.82946787e-01,  9.90052292e-01,
       -3.67044777e+03, -3.66319978e+03,  7.38289554e+03,  7.48281553e+03,
        8.61000000e+02,  7.41612512e+03,  1.25325348e+00,  1.10248592e+00,
        4.98745352e-02,  8.45147096e-03,  8.63560095e-02,  9.00000000e-01,
        4.48041192e-01,  5.00000000e-02,  9.26516415e-02,  8.00000000e-02,
        4.63628755e-02,  

We get a $\chi^2$-value of 14.496 (df = 7, p = 0.043), a RMSEA of 0.05 with a 90%
CI of [0.008, 0.086], a CFI of 0.989, and an SRMR of 0.036. In terms of goodness-of-fit statistics, we do not see much of a difference compared to the baseline model.
Let us do some model comparison. Since the marginalization model is nested in
the basemodel, we can apply the following $\chi^2$-difference test. Note that a **significant**
**result suggests that the higher parametrized model fits significantly worse**.

In [12]:
print(ro.r("anova(fitMarg, fitBase1)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
         Df    AIC    BIC   Chisq Chisq diff Df diff Pr(>Chisq)  
fitBase1  6 7377.7 7482.4  7.3214                                
fitMarg   7 7382.9 7482.8 14.4960     6.4063       1    0.01137 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



Even though significant, the AIC/BIC barely differ across the two groups,
and from the fit indices above, we learned that both models fit well. Thus, the
**marginalization model may be an attractive option to go with**.  
    
#### Didn't get it? No worries!

The alternative approach presented here may be quite confusing as it wasn't covered in the lecture. So lets put it in words.  

The goal here is to investigate the between-group measurement (in)variance of *one specific* indicator. In the approach mentioned before (and in the lecture) we investigated whether measurement invariance holds for *all* indicators.  

As you might have guessed, the indicator we are investigating in this alternative approach is `SP`. Lets revisit the steps:

1. We fit a model in which all loadings but the one from `SP` are fixed (`FitBase1`).

2. We fit alternative models with alternative properties of the `SP` laodings (all the other loadings stay the same). (`GP_model2`: Fix `SP` loading for women but not for men; `fitMarg`: Fix all loadings).

3. We compare model fits. Lets check the comparison of `fitMarg`and `fitBase1` again:

In [13]:
print(ro.r("anova(fitMarg, fitBase1)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
         Df    AIC    BIC   Chisq Chisq diff Df diff Pr(>Chisq)  
fitBase1  6 7377.7 7482.4  7.3214                                
fitMarg   7 7382.9 7482.8 14.4960     6.4063       1    0.01137 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



Before we look at the values, lets think about what this comparison is good for. In the `fitBase1` model we allow the loadings of `SP` to vary between men and women. In the `fitMarg` model we assume them to be equal for both groups. Now we can apply our 'normal' principle again: If the more restrictive model that assumes measurement invariance (`fitMarg`) **is not significantly worse** than the model that allows the loadings to vary (`fitBase1`) **we assume measurement invariance**. The critical point here is that we only investigated measurement invaraince for *one* indicator (`SP`) and not for the whole model (as done in the previous section).
